## 데이터 불러오기

In [1]:
import json
import os

def read_json_file(fp):
    with open(fp, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def load_json_files(dir):
    json_data_list = []

    for filename in os.listdir(dir):
        if filename.endswith('.json'):
            fp = os.path.join(dir, filename)
            json_data = read_json_file(fp)
            json_data_list.extend(json_data)
    
    return json_data_list

files_path = './absa_data/labeling_data'

In [2]:
all_json_data = load_json_files(files_path)

## 화장품 관련 데이터 개수 확인

In [3]:
len(all_json_data)

49996

In [ ]:
all_json_data[0]

- source: 쇼핑몰, SNS
- MainCategory: 화장품 분류(스킨케어, 헤어바디케어, 메이크업뷰티소품, 남성화장품)

In [5]:
from collections import Counter

In [6]:
category_aspect_counts = {}

for entry in all_json_data:
    main_category = entry['MainCategory']
    aspects = entry['Aspects']

    if main_category not in category_aspect_counts:
        category_aspect_counts[main_category] = Counter()

    aspect_texts = [aspect['Aspect'] for aspect in aspects]
    category_aspect_counts[main_category].update(aspect_texts)

In [7]:
# 결과 출력
for main_category, aspect_counts in category_aspect_counts.items():
    print(f"MainCategory: {main_category}")

    # Aspect를 오름차순으로 정렬
    sorted_aspects = sorted(aspect_counts.items(), key=lambda x: x[0])
    
    for aspect, count in sorted_aspects:
        print(f"  {aspect}: {count}")
    print("\n")

MainCategory: 스킨케어
  가격: 4668
  기능/효과: 11421
  디자인: 93
  밀착력/접착력: 73
  발림성: 2989
  보습력/수분감: 9704
  사용감: 543
  색상: 151
  성분: 773
  용기: 644
  용량: 2128
  유통기한: 618
  윤기/피부(톤): 3435
  자극성: 2903
  제품구성: 2645
  제형: 1835
  지속력: 1009
  커버력: 79
  탄력: 978
  편의성/활용성: 3438
  품질: 672
  피부타입: 1353
  향: 2933
  흡수력: 5621


MainCategory: 헤어/바디케어
  가격: 4775
  거품력: 905
  그립감: 9
  기능/효과: 4214
  두피보호: 353
  디자인: 108
  머릿결관리: 3576
  밀착력/접착력: 80
  발림성: 1110
  발색력: 889
  보습력/수분감: 3706
  분사력: 159
  사용감: 375
  색상: 398
  성분: 663
  세정력: 1183
  세팅력/고정력: 1019
  스타일링효과: 1145
  염색력: 2508
  용기: 859
  용량/사이즈: 1910
  유통기한: 536
  윤기/피부(톤): 199
  이염: 371
  자극성: 2052
  제품구성: 1322
  제형: 1349
  지속력/유지력: 1697
  청량감/쿨링감: 1237
  클렌징/제거력: 367
  탈모개선: 1170
  편의성/활용성: 3671
  품질: 726
  피부타입: 367
  향/냄새: 7147
  흡수력: 1761


MainCategory: 메이크업/뷰티소품
  가격: 3753
  기능/효과: 3521
  디자인: 465
  밀착력/접착력: 2072
  발림성: 3711
  발색력: 2639
  보습력/수분감/쿨링감: 3664
  사용감: 297
  사이즈/두께: 837
  색상: 3593
  성분: 173
  용기: 1309
  용량/개수: 1139
  유통기한: 408
  윤기/피부(톤)

## 각 감성 극성 개수 살펴보기

In [11]:
# 각 SentimentPolarity 값의 개수를 저장할 딕셔너리 초기화
sentiment_counts = {'-1': 0, '0': 0, '1': 0}

for data in all_json_data:
    aspects = data.get('Aspects', [])
    for aspect in aspects:
        sentiment_polarity = aspect.get('SentimentPolarity', None)
        
        # SentimentPolarity 값이 존재하면 개수 증가
        if sentiment_polarity in sentiment_counts:
            sentiment_counts[sentiment_polarity] += 1

print("SentimentPolarity Counts:")
for sentiment, count in sentiment_counts.items():
    print(f"{sentiment}: {count}")

SentimentPolarity Counts:
-1: 48082
0: 10184
1: 126218


In [12]:
from collections import defaultdict

main_category_sentiment_counts = defaultdict(lambda: {'-1': 0, '0': 0, '1': 0})

for data in all_json_data:
    main_category = data.get('MainCategory', None)
    
    if main_category:
        # 각 데이터의 Aspects에서 SentimentPolarity 값을 확인
        aspects = data.get('Aspects', [])
        for aspect in aspects:
            sentiment_polarity = aspect.get('SentimentPolarity', None)
            
            if sentiment_polarity in main_category_sentiment_counts[main_category]:
                main_category_sentiment_counts[main_category][sentiment_polarity] += 1

print("MainCategory SentimentPolarity Counts:")
for main_category, sentiment_counts in main_category_sentiment_counts.items():
    print(f"MainCategory: {main_category}")
    for sentiment, count in sentiment_counts.items():
        print(f"  {sentiment}: {count}")
    print("\n")


MainCategory SentimentPolarity Counts:
MainCategory: 스킨케어
  -1: 14377
  0: 4355
  1: 41974


MainCategory: 헤어/바디케어
  -1: 14558
  0: 3316
  1: 36042


MainCategory: 메이크업/뷰티소품
  -1: 14074
  0: 1964
  1: 35340


MainCategory: 남성화장품
  -1: 5073
  0: 549
  1: 12862




## 데이터 전처리 테스트

- 불용어 처리

In [2]:
import re

In [5]:
text = '촉촉하고 사용감이 너무 무겁지 않아서 좋아요 ㅎㅎ 향기도 그렇게 진하지 않고 은은한 장미향이 납니다 ! 그냥 추천순에 뜨길래 구매한건데 좋아서 후기씁니다 ㅎㅎ'
text = re.sub(r"[^가-힣| |]+", "", text)
cleaned_text = re.sub(' +', ' ', text).rstrip()
print(text)
print(cleaned_text)

촉촉하고 사용감이 너무 무겁지 않아서 좋아요  향기도 그렇게 진하지 않고 은은한 장미향이 납니다  그냥 추천순에 뜨길래 구매한건데 좋아서 후기씁니다 
촉촉하고 사용감이 너무 무겁지 않아서 좋아요 향기도 그렇게 진하지 않고 은은한 장미향이 납니다 그냥 추천순에 뜨길래 구매한건데 좋아서 후기씁니다
